## RTA Monitor

1. Define the *maximum* number of seconds you would like to monitor the process for 
2. Define any short comments about the process 

> Note that once all heating steps are complete, no more temperature data can be read. The reading process may terminate before the actual specified number of seconds to read for.

In [10]:
# remeber, this number is in seconds!
# The actual reading time may 
READ_FOR = 15*60
COMMENT = "5 mbar Anneal Retuned"

Where should I save plot data?

In [11]:
from pathlib import Path 

data_path = Path("RTA-data")

Great. Here are some useful `python` libraries that we will be using. Please help me import them by clicking the run button next to the cell.

In [12]:
import plotly.graph_objs as go
import h5py
import numpy as np
import matplotlib.pyplot as plt 
from SerialReader import SerialReaderPlotter

s = SerialReaderPlotter("COM8", baudrate=19200, timeout=60)

## Live Plot
The next cell creates a plot window where our temperature versus time trace will go. Every time you run the following cell, a new graph will be created.

In [13]:
fig = go.FigureWidget(data=[])
fig.update_layout(
    template="plotly_white",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (C)",
)

FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'T…

## Begin
When you are ready, click run in the following cell to start the monitoring process.

In [14]:
"""
In this cell, I left to you the convenience of allowing to other things 
while the serial reader is reading.
"""
s.start_reading(READ_FOR)
# fig.update_layout(xaxis_range=[0,READ_FOR])
s.start_writing(fig)
s.writer.join()

(Reader   ) Starting
(Plotter  ) Starting
(Plotter  ) Exiting


## Saving Data

In [15]:
from datetime import datetime
date = datetime.now().strftime("%Y_%m_%d-%I-%M_%p")
print("Today: {}".format(date))

Today: 2021_08_24-02-48_PM


In [16]:
for i, scatter in enumerate(fig.data):
    filename = data_path/Path("RTA_Run_{}_".format(i) + date + ".h5")

    hf = h5py.File(filename, 'w')
    hf.create_dataset('Time', data=np.array(scatter.x[1:], dtype=float)) 
    hf.create_dataset('Temperature', data=np.array(scatter.y[1:], dtype=float))
    hf.create_dataset("Comment", data = [COMMENT, ])
    hf.close()

## Loading Data

In [17]:
previous_runs = {f.name: h5py.File(f, "r") for f in data_path.glob("*.h5")}

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'RTA-data\RTA-data\RTA_Run_0_2021_08_24-01-19_PM.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
read = go.FigureWidget(data=[])
read.update_layout(
    template="plotly_white",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (C)",
)

for k in previous_runs.keys():
    read.add_scatter(x=[], 
                    y=[], 
                    line=dict(width=1),
                    name = previous_runs[k]["Comment"][0].decode("utf-8"))

    read_dat = read.data[-1]

    with read.batch_update():
        read_dat.x += tuple(previous_runs[k]["Time"][:])
        read_dat.y += tuple(previous_runs[k]["Temperature"][:]) 
                
read

FigureWidget({
    'data': [{'line': {'width': 1},
              'name': 'Vacuum Anneal #1',
              'ty…